In [1]:
%pwd

'/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf/research'

In [2]:
import os

os.chdir('../')
%pwd

'/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf'

#### Base Model Entity

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_model_path: Path
    params_image_size: list
    params_include_top: bool
    params_weights: str
    # params_learning_rate: float
    params_classes: int

In [4]:
from cnn_classifier.constants import *
from cnn_classifier.utils.utilities import read_yaml, create_directories

#### Base Model Configuration

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_base_model_config(self) -> BaseModelConfig:
        config = self.config.base_model
        create_directories([config.root_dir])

        base_model_config = BaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_model_path= Path(config.updated_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            # params_learning_rate=self.params.LEARNING_RATE,
            params_classes=self.params.CLASSES
        )
        return base_model_config

#### Base Model Component

In [6]:
import tensorflow as tf

/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf/cnn_venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [ ]:
class BaseModel:
    def __init__(self, config: BaseModelConfig):
        self.config=config
    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(
            path=self.config.base_model_path,
            model=self.model
        )
    
    @staticmethod
    def define_full_model(model, classes, freez_all, freeze_till):
        if freez_all:
            for layer in model.layers:
                model.trainable = False
        if (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
            
        flatten_layer = tf.keras.layers.Flatten()(model.output)
        output_layer = tf.keras.layers.Dense(
            units=classes,
            activation='softmax'
        )(flatten_layer)

        full_model = tf.keras.Model(
            inputs=model.input,
            outputs=output_layer
        )

        # full_model.compile(
        #     optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        #     loss=tf.keras.losses.CategoricalCrossentropy(),
        #     metrics=["accuracy"]
        # )
        full_model.summary()

        return full_model
    
    def update_base_model(self):
        self.full_model = self.define_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freez_all=True,
            freeze_till=None
            # learning_rate=self.config.params_learning_rate
        )

        self.save_model(
            path=self.config.updated_model_path,
            model=self.full_model
        )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        return model.save(path)

#### Model definition Pipeline

In [8]:
try:
    config = ConfigurationManager()
    base_model_confg = config.get_base_model_config()
    base_model = BaseModel(config=base_model_confg)
    base_model.get_base_model()
    base_model.update_base_model()
except Exception as e:
    raise e

[2025-12-22 16:58:45,930]: cnnClassiferLogger - INFO - utilities - yaml file '<_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfully!'
[2025-12-22 16:58:45,932]: cnnClassiferLogger - INFO - utilities - yaml file '<_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully!'
[2025-12-22 16:58:45,933]: cnnClassiferLogger - INFO - utilities - Created directory at artifacts
[2025-12-22 16:58:45,934]: cnnClassiferLogger - INFO - utilities - Created directory at artifacts/base_model
[2025-12-22 16:58:46,152]: absl - WARNING - saving_api - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │       100,356 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,815,044 (56.51 MB)

 Trainable params: 100,356 (392.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

[2025-12-22 16:58:46,194]: absl - WARNING - saving_api - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
